In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
# xlrd is required to read Excel files
# pip install xlrd>=2.0.1

In [3]:
filename = '../data/titanic3.xls'

df = pd.read_excel(filename)
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


# Beyond 1

Create a series (`most_common_destinations`) in which the index contains the unique values from the `embarked` column, and the values are the most common destination for each value of `embarked`.

In [4]:
most_common_destinations = Series([], dtype=object)

for embarked_value in df['embarked'].dropna().unique():
    most_common_destinations.loc[embarked_value] = df.loc[df['embarked']==embarked_value, 'home.dest'].value_counts().index[0]
    
most_common_destinations

S           New York, NY
C           New York, NY
Q    Ireland Chicago, IL
dtype: object

`home.dest` 특성에 포함된 값들이 반드시 거주지/목적지 형식을 갖지는 않음. 

In [5]:
df['home.dest'].unique()[:5]

array(['St Louis, MO', 'Montreal, PQ / Chesterville, ON', 'New York, NY',
       'Hudson, NY', 'Belfast, NI'], dtype=object)

`'St Louis, MO'`의 경우처럼 주어진 값이 거주지인지 목적지인지 데이터만으로는 판단할 수 없다.
그런 경우엔 필요에 따라 추가 조사를 통해 확인해야한다.

아래 코드는 목적지가 명확한 데이터만 따로 추출하여 `'dest'` 특성으로 추가한다.

In [6]:
df['dest'] = df['home.dest'].apply(
        lambda x: x.split('/', 1)[1].strip() if isinstance(x, str) and '/' in x else np.nan
    )

위 코드 대신 아래 코드 사용 가능.
단 시리즈 객체의 `str` 속성은 문자열로 변환된 값들로 구성된, 시리즈와 유사한 객체를
가리키며, 동시에 문자열을 다루는 다양한 문자열 메서드를 제공한다.

In [7]:
df['dest'] = df['home.dest'].str.split('/').str[1].str.strip()

목적지가 명시된 데이터는 총 171개에 불과하다.

In [8]:
df['dest'].notnull().sum()

171

새로 생성된 `'dest'` 열의 정보만을 이용할 경우 승선항구별 목적지의 최빈값이 달라진다.

In [9]:
most_common_destinations = Series([], dtype=object)

for embarked_value in df['embarked'].dropna().unique():
    most_common_destinations.loc[embarked_value] = df.loc[df['embarked']==embarked_value, 'dest'].value_counts().index[0]
    
most_common_destinations

S          Akron, OH
C    Cooperstown, NY
Q         Bangor, ME
dtype: object

하지만 위 결과의 신뢰도는 아무도 보장할 수 없음에 주의한다.
이유는 `'dest'` 열을 새로 생성하는 과정에서 `'주거지/목적지'` 형식이 아닌 데이터는 제거되었기 때문이다.

그렇다고 아래 코드에서처럼 주거지 또는 목적지 하나만 있는 경우를 모두 목적지로 간주하는 것 또한 바람직하지 않다.
이유는 `'St Louis, MO'`의 경우처럼 주어진 값이 거주지인지 목적지인지 데이터만으로는 판단할 수 없기 때문이다.

In [10]:
df['dest'] = df['home.dest'].apply(
        lambda x: x.split('/', 1)[1].strip() if isinstance(x, str) and '/' in x else x
    )

또는

In [11]:
df['dest'] = df['home.dest'].str.split('/').str[-1].str.strip()

In [12]:
most_common_destinations = Series([], dtype=object)

for embarked_value in df['embarked'].dropna().unique():
    most_common_destinations.loc[embarked_value] = df.loc[df['embarked']==embarked_value, 'dest'].value_counts().index[0]
    
most_common_destinations

S           New York, NY
C           New York, NY
Q    Ireland Chicago, IL
dtype: object

# Beyond 2

Now replace `NaN` values in the `home.dest` coumn with values from `embarked`.  

In [13]:
df['home.dest'] = df['home.dest'].fillna(df['embarked'])

# Beyond 3

Now use the `most_common_destinations` series to replace values in `home.dest` with the most common values for each embarkation point.

In [14]:
df['home.dest'] = df['home.dest'].replace(most_common_destinations)